In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def tanh(z):
    return np.tanh(z)

def standarize(data, train_mean, std):
    return (data - train_mean) / std

In [ ]:
def initialize_parameters(input_dimension, hidden_dimension):
    weight_input = np.random.randn(hidden_dimension, input_dimension + hidden_dimension)
    bias_input = np.zeros((hidden_dimension, 1))

    weight_forget = np.random.randn(hidden_dimension, input_dimension + hidden_dimension)
    bias_forget = np.zeros((hidden_dimension, 1))

    weight_output = np.random.randn(hidden_dimension, input_dimension + hidden_dimension)
    bias_output = np.zeros((hidden_dimension, 1))

    weight_cell = np.random.randn(hidden_dimension, input_dimension + hidden_dimension)
    bias_cell = np.zeros((hidden_dimension, 1))

    parameters = [
        weight_input, bias_input,
        weight_forget, bias_forget,
        weight_output, bias_output, 
        weight_cell, bias_cell
        ]

    return parameters

In [ ]:
def forward_propagation(x, parameters):
    weight_input, bias_input, weight_forget, bias_forget,weight_output, bias_output, weight_cell, bias_cell = parameters
    hidden_states = np.zeros((weight_forget.shape[0], 1))
    cell_states = np.zeros((weight_forget.shape[0], 1))

    sequence_length, input_dim = x.shape
    hidden_dim = weight_forget.shape[0]

    cache = {
        'hidden_states': np.zeros((sequence_length, hidden_dim, 1)),
        'cell_states': np.zeros((sequence_length, hidden_dim, 1)),
        'input_gate': np.zeros((sequence_length, hidden_dim, 1)),
        'forget_gate': np.zeros((sequence_length, hidden_dim, 1)),
        'output_gate': np.zeros((sequence_length, hidden_dim, 1)),
        'cell_candidate': np.zeros((sequence_length, hidden_dim, 1)),
        'x': x.reshape(sequence_length, input_dim, 1)
    }

    for t in range(x.shape[0]):
        x_t = x[t].reshape(-1, 1)
        concat = np.vstack((hidden_states, x_t))

        input_gate = sigmoid(np.dot(weight_input, concat) + bias_input)
        forget_gate = sigmoid(np.dot(weight_forget, concat) + bias_forget)
        output_gate = sigmoid(np.dot(weight_output, concat) + bias_output)
        cell_candidate = sigmoid(np.dot(weight_cell, concat) + bias_cell)

        cell_states = forget_gate * cell_states + input_gate * cell_candidate
        hidden_states = output_gate * tanh(cell_states)

        cache['hidden_states'][t] = hidden_states
        cache['cell_states'][t] = cell_states
        cache['input_gate'][t] = input_gate
        cache['forget_gate'][t] = forget_gate
        cache['output_gate'][t] = output_gate
        cache['cell_candidate'][t] = cell_candidate

    return hidden_states, cell_states, cache

In [ ]:
def backward_propagation_through_time(x, parameters, cache):
    weight_input, bias_input, weight_forget, bias_forget,weight_output, bias_output, weight_cell, bias_cell = parameters

    d_weight_input, d_bias_input = np.zeros_like(weight_input), np.zeros_like(bias_input)
    d_weight_forget, d_bias_forget = np.zeros_like(weight_forget), np.zeros_like(bias_forget)
    d_weight_output, d_bias_output = np.zeros_like(weight_output), np.zeros_like(bias_output)
    d_weight_cell, d_bias_cell = np.zeros_like(weight_cell), np.zeros_like(bias_cell)
    
    d_hidden_next = np.zeros_like(cache['hidden_states'][0])
    d_cell_next = np.zeros_like(cache['cell_states'][0])

    for t in reversed(range(x.shape[0])):
        hidden_states_t = cache['hidden_states'][t]
        cell_states_t = cache['cell_states'][t]
        input_gate_t = cache['input_gate'][t]
        forget_gate_t = cache['forget_gate'][t]
        output_gate_t = cache['output_gate'][t]
        cell_cadidates_t = cache['cell_states'][t]
        x_t = cache['x'][t]

        

In [3]:
file_path = '../../Datasets/조업편차분석.csv'
data = pd.read_csv(file_path)

data['Origin'] = data.index
shuffled_data = data.sample(frac=1, random_state=42).reset_index(drop=True)

train_size = int(len(shuffled_data) * 0.8)
train_data, test_data = shuffled_data.iloc[:train_size], shuffled_data.iloc[train_size:]

x_train, y_train = train_data.drop(columns=['No.', 'A1', 'Origin'], axis=1), train_data['A1']

x_train_mean = np.mean(x_train, axis=0)
x_train_std = np.std(x_train, axis=0)
y_train_mean = np.mean(y_train, axis=0)
y_train_std = np.std(y_train, axis=0)

x_train = standarize(x_train, x_train_mean, x_train_std).to_numpy()
y_train = standarize(y_train, y_train_mean, y_train_std).to_numpy().reshape(-1, 1)

test_data_sorted = test_data.sort_values(by='Origin').reset_index(drop=True)

x_test_sorted, y_test_sorted = test_data_sorted.drop(columns=['No.', 'A1', 'Origin'], axis=1), test_data_sorted['A1']
x_test_sorted = standarize(x_test_sorted, x_train_mean, x_train_std).to_numpy()
y_test_sorted = standarize(y_test_sorted, y_train_mean, y_train_std).to_numpy().reshape(-1, 1)

In [5]:
input_dimension = 53
hidden_dimension = 64
output_dimension = 1